In [1]:
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import tf2onnx
import onnxruntime
from matplotlib import pyplot as plt
from collections import Counter
from tqdm.auto import tqdm

2024-11-15 10:11:09.646017: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 10:11:09.646398: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 10:11:09.648841: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 10:11:09.655211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731661869.665745  178544 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731661869.66

In [2]:
data = pd.read_csv("train.csv", header=None)
display(data)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.1,e-399,NaN,w-219,NaN,NaN,w-77,w-71,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-215,1,0,0,0
1,0.3,e-399,NaN,w-219,NaN,NaN,w-77,w-71,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-215,1,0,0,0
2,0.6,e-397,NaN,w-215,NaN,NaN,w-77,w-73,w-72,w-109,...,w-197,w-185,NaN,NaN,NaN,w-213,1,0,0,0
3,0.8,e-396,NaN,w-214,NaN,NaN,w-79,w-72,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-213,1,0,0,0
4,1.0,e-394,NaN,w-211,NaN,NaN,w-81,w-70,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-211,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12163,-4.3,w-185,d-322,w-127,w-130,w-187,NaN,NaN,NaN,e-153,...,w-28,w-30,w-35,w-44,w-43,w-73,0,1,1,0
12164,-4.4,w-187,d-327,w-133,w-134,w-189,NaN,NaN,NaN,e-150,...,w-28,w-30,w-32,w-41,w-47,w-64,0,1,1,0
12165,-4.5,w-136,d-331,w-144,w-137,w-191,NaN,NaN,NaN,e-148,...,w-31,w-29,w-33,w-40,w-51,w-56,0,1,1,0
12166,-4.6,w-96,d-336,w-152,w-137,w-193,NaN,NaN,NaN,e-145,...,w-35,w-32,w-31,w-37,w-53,w-59,0,1,1,0


In [3]:
def parse_sensor(s, letters, max_val=501.0):
    if not s:
        return 1.0
    if s[0] not in letters:
        return 1.0
    return float(s[2:])/max_val

def parse_keys(keys):
    """
    up,down,left,right flags to one-hot encoded representation
    (up+left)   -> 0 (up)      -> 1 (up+right)   -> 2
    (left)      -> 3 (nothing) -> 4 (right)      -> 5
    (down+left) -> 6 (down)    -> 7 (down+right) -> 8
    """
    if np.array_equal(keys, [1,0,1,0]):
        return np.array([1.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [1,0,0,0]):
        return np.array([0.0,1.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [1,0,0,1]):
        return np.array([0.0,0.0,1.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    
    if np.array_equal(keys, [0,0,1,0]):
        return np.array([0.0,0.0,0.0, 1.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [0,0,0,0]) or np.array_equal(keys, [0,0,1,1]):
        return np.array([0.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [0,0,0,1]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,1.0, 0.0,0.0,0.0])

    if np.array_equal(keys, [0,1,1,0]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 1.0,0.0,0.0])
    if np.array_equal(keys, [0,1,0,0]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,1.0,0.0])
    if np.array_equal(keys, [0,1,0,1]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,1.0,0.0])

    raise ValueError(f"Unknown key combination: {keys}")
    

X = []
Y = []
with open("train.csv") as f:
    for row in tqdm(csv.reader(f)):
        velocity = np.array([float(row[0])])
        sensors = row[1:25]
        keys = np.fromiter((float(k) for k in row[25:]), dtype=float)
        keys = parse_keys(keys)
        walls = np.fromiter((parse_sensor(s, ("w",)) for s in sensors), dtype=float)
        enemy = np.fromiter((parse_sensor(s, ("e",)) for s in sensors), dtype=float)
        diamonds = np.fromiter((parse_sensor(s, ("d",)) for s in sensors), dtype=float)
        X.append(np.concatenate((velocity, walls, enemy, diamonds)))
        Y.append(keys)

X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape

12168it [00:00, 38301.96it/s]


((12168, 73), (12168, 9))

In [4]:
def create_ff_model():
    input = tf.keras.Input(shape=(X.shape[1],), name="input")
    hidden = tf.keras.layers.Dense(64, activation="tanh", name="hidden1")(input)
    output = tf.keras.layers.Dense(Y.shape[1], activation='softmax', name='output')(hidden)

    model = tf.keras.Model(name="ff_model", inputs=input, outputs=output)
    model.compile(optimizer='adam', loss="categorical_crossentropy")
    return model

ff_model = create_ff_model()

2024-11-15 10:11:11.614169: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
ff_model.summary()

Model: "ff_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 73)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden1 (Dense)                 │ (None, 64)             │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,321 (20.79 KB)

 Trainable params: 5,321 (20.79 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
ff_model.fit(X, Y, epochs=20, validation_split=0.2)

Epoch 1/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 975us/step - loss: 1.7636 - val_loss: 1.6157
Epoch 2/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 1.4751 - val_loss: 1.5662
Epoch 3/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 1.4380 - val_loss: 1.5925
Epoch 4/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 1.3966 - val_loss: 1.5768
Epoch 5/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - loss: 1.3788 - val_loss: 1.5809
Epoch 6/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 1.3575 - val_loss: 1.5690
Epoch 7/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - loss: 1.3500 - val_loss: 1.6110
Epoch 8/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - loss: 1.3559 - val_loss: 1.5444
Epoch 9/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 1.3283 - val_loss: 1.5304
Epoch 10/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - loss: 1.3368 - val_loss: 1.5819
Epoch 11/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - loss: 1.3135 - val_loss: 1.5266
Epoch 12/20
305/305 ━━━━━━━━━━

In [7]:
tf.keras.models.save_model(ff_model, "../models/ff-one-hot.keras")

In [8]:
import random
random.seed(42)

test_input = np.array([[random.random() for _ in range(73)]])

In [9]:
[float(f) for f in ff_model(test_input)[0]]

[0.0075528561137616634,
 0.010143924504518509,
 0.026259252801537514,
 0.06125572323799133,
 0.035694923251867294,
 0.363323450088501,
 0.1331682801246643,
 0.36258721351623535,
 1.4340771485876758e-05]

In [10]:
spec = (tf.TensorSpec((None, X.shape[1]), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(ff_model, input_signature=spec, opset=13, output_path="../models/ff-one-hot.onnx")

I0000 00:00:1731661876.698218  178544 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1731661876.698298  178544 single_machine.cc:361] Starting new session
I0000 00:00:1731661876.720339  178544 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1731661876.720422  178544 single_machine.cc:361] Starting new session


In [11]:
onnx_session = onnxruntime.InferenceSession("../models/ff-one-hot.onnx")

In [12]:
[float(f) for f in onnx_session.run(["output"], {"input": test_input.astype(np.float32)})[0][0]]

[0.00755285331979394,
 0.010143925435841084,
 0.026259256526827812,
 0.06125573068857193,
 0.035694923251867294,
 0.363323450088501,
 0.1331682801246643,
 0.36258718371391296,
 1.4340770576382056e-05]